# Pytorch Efficientnet + ArcFace [training]
## Introduction
* This notebook uses Efficient net and ArcFace on pytorch for training a model.
* to train for 10 epoch I got validation accuracy about 0.25 and validation GAP score about 0.20
* Inference notebook for submission is another one https://www.kaggle.com/zaccheroni/pytorch-efficientnet-arcface-submission Here!
* To tell the truth I'm a beginer, so if there are any mistakes, please tell me!

I used a notebook https://www.kaggle.com/rhtsingh/pytorch-training-inference-efficientnet-baseline by @rhtsingh - as a reference.

In [1]:
# !mkdir -p /tmp/pip/cache/
# !cp ../input/resources-for-google-landmark-recognition-2020/efficientnet_pytorch-0.6.3-py3-none-any.whl /tmp/pip/cache/
# !pip install --no-index --find-links /tmp/pip/cache/ efficientnet_pytorch

In [2]:
# import efficientnet_pytorch

In [ ]:
# if you need notification
#!pip install slackweb
# import slackweb

In [ ]:
import os
import gc
gc.enable()
import sys
import math
import json
import time
import random
from glob import glob
from datetime import datetime

import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import multiprocessing
from sklearn.preprocessing import LabelEncoder

import torch
import torchvision
from torch import Tensor
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler
from tqdm import tqdm

import albumentations as A

import sklearn

import warnings
warnings.filterwarnings("ignore")

import csv
import pprint
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
import math

from sklearn.model_selection import train_test_split
import torch.optim as optim

In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import pandas as pd
import requests
import io
import urllib.parse
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import functools
from PIL import Image  # Image utilities.
import tqdm
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import imageio as io_temp
from skimage.transform import resize
from sklearn.metrics import roc_curve, auc, roc_auc_score, classification_report, confusion_matrix


cudnn.benchmark = True
plt.ion()   # interactive mode

In [ ]:
# Seed everything to avoid non-determinism.
def seed_everything(seed=2020):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
seed_everything()

In [3]:
SOURCE_URL = 'https://storage.googleapis.com/dm-turtle-recall/images.tar'
IMAGE_DIR = './turtle_recall/images'
TAR_PATH = os.path.join(IMAGE_DIR, os.path.basename(SOURCE_URL))
EXPECTED_IMAGE_COUNT = 13891

%sx mkdir --parents "{IMAGE_DIR}"
if len(os.listdir(IMAGE_DIR)) != EXPECTED_IMAGE_COUNT:
  %sx wget --no-check-certificate -O "{TAR_PATH}" "{SOURCE_URL}"
  %sx tar --extract --file="{TAR_PATH}" --directory="{IMAGE_DIR}"
  %sx rm "{TAR_PATH}"

print(f'The total number of images is: {len(os.listdir(IMAGE_DIR))}')

In [10]:
os.listdir(IMAGE_DIR)

In [9]:
BASE_URL = 'https://storage.googleapis.com/dm-turtle-recall/'


def read_csv_from_web(file_name):
  url = urllib.parse.urljoin(BASE_URL, file_name)
  content = requests.get(url).content
  return pd.read_csv(io.StringIO(content.decode('utf-8')))

# Read in csv files.
# train = read_csv_from_web('train.csv')
test = read_csv_from_web('test.csv')
# extra = read_csv_from_web('extra_images.csv')
# sample_submission = read_csv_from_web('sample_submission.csv')

# Convert image_location strings to lowercase.
for df in [test]:
  df.image_location = df.image_location.apply(lambda x: x.lower())
  assert set(df.image_location.unique()) == set(['left', 'right', 'top'])

In [ ]:
# ls = pd.unique(train['turtle_id'])
# classes = {}
# for i in range(len(ls)):
#     classes[ls[i]] = i
# class_names = classes
# num_classes = len(class_names)

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(train[['image_id','image_location']], train['turtle_id'], test_size=0.20, random_state=42, stratify=train['turtle_id'])
# X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.50, random_state=42, stratify=y_test)

# print(X_train.shape, X_test.shape, X_val.shape ,y_train.shape, y_test.shape, y_val.shape)

# train_ds = pd.concat([X_train, y_train], axis=1)
# train_ds['type'] = "train"
# test_ds = pd.concat([X_test, y_test], axis=1)
# test_ds['type'] = "test"
# val_ds = pd.concat([X_val, y_val], axis=1)
# val_ds['type'] = "val"

In [ ]:
dataset_sizes = {'train': len(train) + len(extra)}

In [ ]:
dataset = pd.concat([train,extra], axis=0)

In [ ]:
dataset['image_id'] = IMAGE_DIR + "/" + dataset['image_id'].astype(str) + ".JPG"

In [ ]:
dataset = dataset[dataset['turtle_id'].isin(train['turtle_id'])]
dataset = dataset.reset_index()
dataset.pop('index')
dataset.head()

In [ ]:
# dataset['turtle_id'] = dataset['turtle_id'].map(classes)

In [ ]:
class TurtleDataset(Dataset):
    """Turtle dataset."""

    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.csv = csv_file
        self.transform = transform

    def __len__(self):
        return len(self.csv)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = self.csv.iloc[idx, 0]
        image = io_temp.imread(img_name)
        details = self.csv.iloc[idx, 1:]
        details = np.array([details])
        details = details[:1]
        sample = {'image': image, 'image_orientation': details[0][0], 'turtle_id': details[0][1]}

        if self.transform:
            sample = self.transform(sample)

        return sample

In [ ]:
class Rescale(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size = 224):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, turtle_id = sample['image'], sample['turtle_id'] # , sample['image_orientation']
        
        # ADD CODE EITHER HERE

        h, w = image.shape[:2]
        crop_size = min(w, h)
        crop = image[(h - crop_size) // 2 : (h + crop_size) // 2, (w - crop_size) // 2 : (w + crop_size) // 2]
        img = resize(crop, (self.output_size, self.output_size))

        # OR ADD CODE HERE
        
        return [img.transpose((2,0,1)).astype(np.double),turtle_id]

In [ ]:
class Rescale_test(object):
    """Rescale the image in a sample to a given size.

    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size = 224):
        assert isinstance(output_size, (int, tuple))
        self.output_size = output_size

    def __call__(self, sample):
        image, turtle_id = sample['image'], sample['turtle_id'] # , sample['image_orientation']
        
        # ADD CODE EITHER HERE

        h, w = image.shape[:2]
        crop_size = min(w, h)
        crop = image[(h - crop_size) // 2 : (h + crop_size) // 2, (w - crop_size) // 2 : (w + crop_size) // 2]
        img = resize(crop, (self.output_size, self.output_size))

        # OR ADD CODE HERE
        
        return [img.transpose((2,0,1)).astype(np.double)]

In [ ]:
import os

In [ ]:
IN_KERNEL = os.environ.get('KAGGLE_WORKING_DIR') is not None
MIN_SAMPLES_PER_CLASS = 30
BATCH_SIZE = 8
NUM_WORKERS = multiprocessing.cpu_count()
MAX_STEPS_PER_EPOCH = 15000
NUM_EPOCHS = 20
LOG_FREQ = 400
NUM_TOP_PREDICTS = 20

In [ ]:
# train = pd.read_csv('../input/landmark-recognition-2020/train.csv')
# test = pd.read_csv('../input/landmark-recognition-2020/sample_submission.csv')
# train_dir = '../input/landmark-recognition-2020/train/'
# test_dir = '../input/landmark-recognition-2020/test/'

In [ ]:
# train, val = train_test_split(train, test_size=0.02)

In [ ]:
# class ImageDataset(torch.utils.data.Dataset):
#     def __init__(self, dataframe: pd.DataFrame, image_dir:str, mode: str):
#         self.df = dataframe
#         self.mode = mode
#         self.image_dir = image_dir
        
#         transforms_list = []
#         if self.mode == 'train':
#             # Increase image size from (64,64) to higher resolution,
#             # Make sure to change in RandomResizedCrop as well.
#             transforms_list = [
#                 transforms.Resize((64,64)),
#                 transforms.RandomHorizontalFlip(),
#                 transforms.RandomChoice([
#                     transforms.RandomResizedCrop(64),
#                     transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
#                     transforms.RandomAffine(degrees=15, translate=(0.2, 0.2),
#                                             scale=(0.8, 1.2), shear=15,
#                                             resample=Image.BILINEAR)
#                 ]),
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                       std=[0.229, 0.224, 0.225]),
#             ]
#         else:
#             transforms_list.extend([
#                 # Keep this resize same as train
#                 transforms.Resize((64,64)),
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                                       std=[0.229, 0.224, 0.225]),
#             ])
#         self.transforms = transforms.Compose(transforms_list)

#     def __getitem__(self, index: int):
#         image_id = self.df.iloc[index].id
#         image_path = f"{self.image_dir}/{image_id[0]}/{image_id[1]}/{image_id[2]}/{image_id}.jpg"
#         image = Image.open(image_path)
#         image = self.transforms(image)

#         if self.mode == 'test':
#             return {'image':image}
#         else:
#             return {'image':image, 
#                     'target':self.df.iloc[index].landmark_id}

#     def __len__(self) -> int:
#         return self.df.shape[0]

In [ ]:
def load_data(train):
    counts = train.turtle_id.value_counts()
    print('train_df', train.shape)

    label_encoder = LabelEncoder()
    label_encoder.fit(train.turtle_id.values)
    print('found classes', len(label_encoder.classes_))

    train.turtle_id = label_encoder.transform(train.turtle_id)

    train_dataset = TurtleDataset(train, transform=transforms.Compose([Rescale(256)]))

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, num_workers=4, drop_last=True)
        
    return train_loader, label_encoder, len(label_encoder.classes_)

In [ ]:
def adam(parameters, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
    if isinstance(betas, str):
        betas = eval(betas)
    return optim.Adam(parameters,
                      lr=lr,
                      betas=betas,
                      eps=eps,
                      weight_decay=weight_decay)

In [ ]:
class AverageMeter:
    ''' Computes and stores the average and current value '''
    def __init__(self) -> None:
        self.reset()

    def reset(self) -> None:
        self.val = 0.0
        self.avg = 0.0
        self.sum = 0.0
        self.count = 0

    def update(self, val: float, n: int = 1) -> None:
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [ ]:
def GAP(predicts: torch.Tensor, confs: torch.Tensor, targets: torch.Tensor) -> float:
    ''' Simplified GAP@1 metric: only one prediction per sample is supported '''
    assert len(predicts.shape) == 1
    assert len(confs.shape) == 1
    assert len(targets.shape) == 1
    assert predicts.shape == confs.shape and confs.shape == targets.shape

    _, indices = torch.sort(confs, descending=True)

    confs = confs.cpu().numpy()
    predicts = predicts[indices].cpu().numpy()
    targets = targets[indices].cpu().numpy()

    res, true_pos = 0.0, 0

    for i, (c, p, t) in enumerate(zip(confs, predicts, targets)):
        rel = int(p == t)
        true_pos += rel

        res += true_pos / (i + 1) * rel

    res /= targets.shape[0] # FIXME: incorrect, not all test images depict landmarks
    return res

In [ ]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, train, label=False):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        if train:
            one_hot = torch.zeros(cosine.size(), device='cuda')
            one_hot.scatter_(1, label.cuda().view(-1, 1).long(), 1)
            output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        else:
            output = cosine
        output *= self.s

        return output

In [ ]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM,self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'

In [ ]:
class EfficientNetEncoderHead(nn.Module):
    def __init__(self, depth, num_classes):
        super(EfficientNetEncoderHead, self).__init__()
        self.depth = depth
        self.base = efficientnet_pytorch.EfficientNet.from_pretrained(f'efficientnet-b{self.depth}')
        self.gem = GeM()
        self.output_filter = self.base._fc.in_features
        self.fc = nn.Linear(self.output_filter, 1000)
        self.arcface = ArcMarginProduct(1000, num_classes)
    def forward(self, x, label):
        x = self.base.extract_features(x)
        x = self.gem(x).squeeze()
        x = self.fc(x)
        if self.training:
            x = self.arcface(x, self.training, label)
        else:
            x = self.arcface(x, self.training)
        return x

## validation
This 'val_step' caliculate a validation accuracy and GAP score

In [ ]:
def val_step(val_loader,
        model,
        criterion,
        label_encoder,
        all_val_count):
    
    val_losses = AverageMeter()
    val_gap_score = AverageMeter()
    val_acc = AverageMeter()
    model.eval()
    acc_count = 0
    first = True
    end = time.time()
    for i, j in val_loader:
        input_ = i
        target = j
        batch_size, _, _, _ = input_.shape
        
        output = model(input_.float().cuda(), target.cuda())
        confs, predicts = torch.max(output.detach(), dim=1)
        
        if first:
            all_confs = confs
            all_predicts = predicts
            all_targets = target
            first = False
        else:
            all_confs = torch.cat([all_confs, confs])
            all_predicts = torch.cat([all_predicts, predicts])
            all_targets = torch.cat([all_targets, target])

    val_gap_score = GAP(all_predicts, all_confs, all_targets)
    val_gap_score = val_gap_score * len(all_confs) / all_val_count
    
    for i, (c, p, t) in enumerate(zip(all_confs, all_predicts, all_targets)):
        if p == t:
            acc_count += 1
                
    acc = float(acc_count) / all_val_count
    val_time = time.time() - end
    return acc, val_gap_score, val_time

## Training

In [ ]:
def train_step(train_loader, 
          model, 
          criterion, 
          optimizer,
          epoch, 
          lr_scheduler):
    print(f'epoch {epoch}')
    batch_time = AverageMeter()
    losses = AverageMeter()
    avg_score = AverageMeter()

    model.train()
    num_steps = min(len(train_loader), MAX_STEPS_PER_EPOCH)

    print(f'total batches: {num_steps}')

    end = time.time()
    lr = None

    for i, j in train_loader:
        input_ = i
        target =j
        batch_size, _, _, _ = input_.shape
        
        output = model(input_.float().cuda(), target.cuda())
        loss = criterion(output, target.cuda())
        confs, predicts = torch.max(output.detach(), dim=1)
        avg_score.update(GAP(predicts, confs, target))
        losses.update(loss.data.item(), input_.size(0))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        lr = optimizer.param_groups[0]['lr']
        
        batch_time.update(time.time() - end)
        end = time.time()

        
    acc, val_gap, val_time = val_step(train_loader, model, criterion, label_encoder, len(dataset))
    print('validation with training data '+str(val_time))
    print(f'time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
            f'loss {losses.val:.4f} ({losses.avg:.4f})\t'
            f'GAP {avg_score.val:.4f} ({avg_score.avg:.4f})\t'
            f'val_acc {acc}\t'
            f'val_GAP {val_gap:.4f}\t'
                 )
            #slack = slackweb.Slack(url="~~~~~~")
            #slack.notify(text= f'{epoch} [{i}/{num_steps}]\t'
                    #f'time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                    #f'loss {losses.val:.4f} ({losses.avg:.4f})\t'
                    #f'GAP {avg_score.val:.4f} ({avg_score.avg:.4f})\t'
                    #f'val_acc {acc}\t'
                    #f'val_GAP {val_gap:.4f}\t')
              

    print(f' * average GAP on train {avg_score.avg:.4f}')
    print(f' time {batch_time.sum:.4f}')
    return avg_score.avg, losses.avg

In [ ]:
def inference(data_loader, model):
    model.eval()

    activation = nn.Softmax(dim=1)
    all_predicts, all_confs, all_targets = [], [], []

    with torch.no_grad():
        for i, data in enumerate(tqdm(data_loader, disable=IN_KERNEL)):
            if data_loader.dataset.mode != 'test':
                input_, target = data['image'], data['target']
            else:
                input_, target = data['image'], None

            output = model(input_.cuda())
            output = activation(output)

            confs, predicts = torch.topk(output, NUM_TOP_PREDICTS)
            all_confs.append(confs)
            all_predicts.append(predicts)

            if target is not None:
                all_targets.append(target)

    predicts = torch.cat(all_predicts)
    confs = torch.cat(all_confs)
    targets = torch.cat(all_targets) if len(all_targets) else None

    return predicts, confs, targets

Before starting training, check if there is a trained model. If I have one, I will load it also with optimizer and scheduler

I made "selected_classes.csv". This file shows which classes are selected, in another word, this model is classifying into these classes.

In [ ]:
if __name__ == '__main__':
    modelname = 'the_model'
    input_dir = '../input/'
    
    global_start_time = time.time()
    train_loader, label_encoder, num_classes = load_data(dataset)

    all_classes = label_encoder.classes_
    all_classes = list(all_classes)
    selected_classes = train.turtle_id
    with open('selected_classes.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(all_classes)
    
    model = EfficientNetEncoderHead(depth=7, num_classes=num_classes)
    model.cuda()

    criterion = nn.CrossEntropyLoss()

    optimizer = adam(model.parameters(), lr=1e-3, betas=(0.9,0.999), eps=1e-3, weight_decay=1e-4)
    scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_loader)*NUM_EPOCHS, eta_min=1e-6)

    s = False
    # if there is 'learning.txt', read it and start training from the epoch which is written in that file.
    if os.path.exists(input_dir + 'learning.txt'):
        with open(input_dir + 'learning.txt') as f:
            s = f.read()
            
    # opttimizer saving dir
    opt_shc_path = 'optimizer_and_scheduler'
        
    
    if s:
        model.load_state_dict(torch.load(input_dir + 'the_model'+s+'.pth'))
        start_epoch = int(s) + 1
        checkpoint = torch.load(input_dir + 'optimizer_and_scheduler')
        optimizer.load_state_dict(checkpoint['optimizer'])
        scheduler.load_state_dict(checkpoint['scheduler'])
        print('optimizer and scheduler are loaded')
        
        pre_history = pd.read_csv(input_dir + 'the_model_history.csv')
    else:
        pre_history = pd.DataFrame(columns=['epoch', 'GAP', 'loss'])
        start_epoch = 1

        
    for epoch in range(start_epoch, NUM_EPOCHS + 1):
        print('-' * 50)
        score, loss = train_step(train_loader, model, criterion, optimizer, epoch, scheduler)
        pre_history = pre_history.append({'GAP':score,'epoch':epoch,'loss':loss}, ignore_index=True)
        
        model_path = 'the_model'+str(epoch)+'.pth'
        state = {
            'optimizer': optimizer.state_dict(),
            'scheduler': scheduler.state_dict()
        }
        torch.save(model.state_dict(), model_path)
        torch.save(state, opt_shc_path)
        
        with open('learning.txt', mode='w') as f:
            f.write(str(epoch))
            
#         acc, val_gap, _ = val_step(val_loader, model, criterion, label_encoder, all_val_count)
        
        pre_history.to_csv('the_model_history.csv')
            
        # if you want to know about learning on slack
        #slack = slackweb.Slack(url="~~~")
        #slack.notify(text= f'{epoch:.4f} epoch finished\t'
        #            f'val_acc {acc}\t'
        #            f'val_GAP {val_gap:.4f}\t')
        
    #slack.notify(text= 'all learning finished')

In [ ]:
def load_data_test(train):
    print('train_df', train.shape)

    train_dataset = TurtleDataset(train, transform=transforms.Compose([Rescale(256)]))

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                              shuffle=True, num_workers=4, drop_last=True)
        
    return train_loader


test_loader = load_data_test(test)

In [ ]:
def test_step(val_loader,
        model):
    
    model.eval()
    for i in val_loader:
        input_ = i
#         batch_size, _, _, _ = input_.shape
        print(i)
        
        output = model(input_[0].float().cuda())
        confs, predicts = torch.max(output.detach(), dim=1)
        break
        
    print(predicts)

    return 

In [ ]:
test_step(train_loader, model)